<a href="https://colab.research.google.com/github/DenCoder618/BabyGAN/blob/master/NewBabyGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://drive.google.com/drive/folders/1xwqqG0HkLe2AiXxjC-XK8OfvMKT1jBlp

In [ ]:
#@title # **Подготавливаем нейросеть** { display-mode: "form" }
#@markdown * Установка всех необходимых компонентов для запуска
#@markdown * Требуеся доступ к гугл диску (подробнее в инструкции)

#@ title # Клонируем репозиторий и устанавливаем все необходимые компоненты { vertical-output: true, display-mode: "form" }
clear = True #@ param {type:"boolean"}
%tensorflow_version 1.x

import os, cv2, math, pickle, imageio, warnings, PIL.Image
import numpy as np
from PIL import Image
import tensorflow as tf
from random import randrange
import moviepy.editor as mpy
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from IPython.display import clear_output
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
%matplotlib inline
warnings.filterwarnings("ignore")

def plot_image(imgB, fs = 10):
  f, axarr = plt.subplots(1,1, figsize=(fs,fs))
  axarr[0].imshow(imgB)
  axarr[0].title.set_text('Фото ребенка')
  plt.show()

def generate_final_images(latent_vector, direction, coeffs, i):
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + coeffs*direction)[:8]
    new_latent_vector = new_latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(new_latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    img_path = "/content/BabyGAN/for_animation/" + str(i) + ".png"
    img.thumbnail(animation_size, PIL.Image.ANTIALIAS)
    img.save(img_path)
    face_img.append(imageio.imread(img_path))
    clear_output()
    return img

def generate_final_image(latent_vector, direction, coeffs):
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + coeffs*direction)[:8]
    new_latent_vector = new_latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(new_latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    img.thumbnail(size, PIL.Image.ANTIALIAS)
    img.save("face.png")
    if download_image == True: files.download("face.png")
    return img

def plot_parents(fs = 10):
  f, axarr = plt.subplots(1,2, figsize=(fs,fs))
  axarr[0].imshow(Image.open('/content/BabyGAN/aligned_images/father_01.png'))
  axarr[0].title.set_text('Фото отца')
  axarr[1].imshow(Image.open('/content/BabyGAN/aligned_images/mother_01.png'))
  axarr[1].title.set_text('Фото матери')
  plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
  plt.show()

def plot_three_images(imgB, fs = 10):
  f, axarr = plt.subplots(1,3, figsize=(fs,fs))
  axarr[0].imshow(Image.open('/content/BabyGAN/aligned_images/father_01.png'))
  axarr[0].title.set_text('Фото отца')
  axarr[1].imshow(imgB)
  axarr[1].title.set_text('Фото ребенка')
  axarr[2].imshow(Image.open('/content/BabyGAN/aligned_images/mother_01.png'))
  axarr[2].title.set_text('Фото матери')
  plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
  plt.show()

!rm -rf sample_data
!git clone https://github.com/DenCoder618/BabyGAN.git
%cd /content/BabyGAN
!mkdir aligned_images data father_image mother_image

import config, dnnlib
import dnnlib.tflib as tflib
from encoder.generator_model import Generator

age_direction = np.load('/content/BabyGAN/ffhq_dataset/latent_directions/age.npy')
horizontal_direction = np.load('/content/BabyGAN/ffhq_dataset/latent_directions/angle_horizontal.npy')
vertical_direction = np.load('/content/BabyGAN/ffhq_dataset/latent_directions/angle_vertical.npy')
eyes_open_direction = np.load('/content/BabyGAN/ffhq_dataset/latent_directions/eyes_open.npy')
gender_direction = np.load('/content/BabyGAN/ffhq_dataset/latent_directions/gender.npy')
smile_direction = np.load('/content/BabyGAN/ffhq_dataset/latent_directions/smile.npy')
print()

if clear:
  clear_output()

if not os.path.isdir('/content/drive/My Drive/BabyGAN'):
  print("Перейдите по ссылке, выберите свой аккаунт")
  print("Разрешите доступ и скопируйте авторизационный код в поле ниже")
  print()

drive.mount('/content/drive')
if clear:
  clear_output()
if os.path.isdir('/content/drive/My Drive/BabyGAN'):
  print("0% --- Copying has started")
  !cp '/content/drive/My Drive/BabyGAN/finetuned_resnet.h5' '/content/BabyGAN/data'
  !cp '/content/drive/My Drive/BabyGAN/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5' '/content/BabyGAN'
  print("50% -- Checkpoints copied")
  !cp '/content/drive/My Drive/BabyGAN/karras2019stylegan-ffhq-1024x1024.pkl' '/content/BabyGAN'
  !cp '/content/drive/My Drive/BabyGAN/vgg16_zhang_perceptual.pkl' '/content/BabyGAN'
  print("75% -- Weights copied")
  !cp '/content/drive/My Drive/BabyGAN/shape_predictor_68_face_landmarks.dat.bz2' '/content/BabyGAN'
  print("100% - Dictionary copied")
  print()
  if clear:
    clear_output()
  print("Нейросеть полностью готова")
else: raise ValueError('Ошибка подключения')

In [ ]:
#@title # **Загружаем лица родителей** { display-mode: "form" }
#@markdown * Сначала загрузите фото отца, потом фото матери
#@markdown * Во время обработки не отключайте экран устройства
#@markdown * Обработка занимает примерно 3 минуты
!rm -rf /content/BabyGAN/father_image/*.*
!rm -rf /content/BabyGAN/mother_image/*.*

# загружаем фото отца
print('Загрузите фото отца:')
uploaded = list(files.upload().keys())
if len(uploaded) > 1: print('Будет использовано только одно изображение')
fat = uploaded[0]
FATHER_FILENAME = "father." + fat.split(".")[-1]
os.rename(fat, FATHER_FILENAME)
father_path = "/content/BabyGAN/father_image/" + FATHER_FILENAME
!mv -f $FATHER_FILENAME $father_path

# загружаем фото матери
print('Загрузите фото матери:')
uploaded = list(files.upload().keys())
if len(uploaded) > 1: raise ValueError('Будет использовано только одно изображение')
mot = uploaded[0]
MOTHER_FILENAME = "mother." + mot.split(".")[-1]
os.rename(mot, MOTHER_FILENAME)
mother_path = "/content/BabyGAN/mother_image/" + MOTHER_FILENAME
!mv -f $MOTHER_FILENAME $mother_path

!python align_images.py /content/BabyGAN/father_image /content/BabyGAN/aligned_images
if os.path.isfile('/content/BabyGAN/aligned_images/father_01.png'):
  pil_father = Image.open('/content/BabyGAN/aligned_images/father_01.png')
  (fat_width, fat_height) = pil_father.size
  resize_fat = max(fat_width, fat_height)/256
  # display(pil_father.resize((int(fat_width/resize_fat), int(fat_height/resize_fat))))
  if clear: clear_output()
else: raise ValueError('Лицо отца не обнаружено')

!python align_images.py /content/BabyGAN/mother_image /content/BabyGAN/aligned_images
if os.path.isfile('/content/BabyGAN/aligned_images/mother_01.png'):
  pil_mother = Image.open('/content/BabyGAN/aligned_images/mother_01.png')
  (mot_width, mot_height) = pil_mother.size
  resize_mot = max(mot_width, mot_height)/256
  # display(pil_mother.resize((int(mot_width/resize_mot), int(mot_height/resize_mot))))
  if clear: clear_output()
else: raise ValueError('Лицо матери не обнаружено')
plot_parents()

#АНАЛИЗ ЛИЦ
print()
print('Запускаем анализ лиц')
print('Это займет примерно 3 минуты')
print()
print('Получение векторов скрытых признаков..')
#@ title # Анализируем лица родителей
#@ markdown * Получение векторов скрытых признаков

#!python encode_images.py --help
!python encode_images.py \
  --early_stopping False \
  --lr=0.25 \
  --batch_size=2 \
  --iterations=100 \
  --output_video=False \
  /content/BabyGAN/aligned_images \
  /content/BabyGAN/generated_images \
  /content/BabyGAN/latent_representations

tflib.init_tf()
URL_FFHQ = "/content/BabyGAN/karras2019stylegan-ffhq-1024x1024.pkl"
with dnnlib.util.open_url(URL_FFHQ, cache_dir=config.cache_dir) as f:
  generator_network, discriminator_network, Gs_network = pickle.load(f)
generator = Generator(Gs_network, batch_size=1, randomize_noise=False)
model_scale = int(2*(math.log(1024,2)-1))

if len(os.listdir('/content/BabyGAN/generated_images')) == 2:
  first_face = np.load('/content/BabyGAN/latent_representations/father_01.npy')
  second_face = np.load('/content/BabyGAN/latent_representations/mother_01.npy')
  print()
  if clear: clear_output()
  print("Генерация векторов завершена! Дальше самое интересное")
else: raise ValueError('Невозможно считать лицо. Выберите другие фото')

In [ ]:
#@title # **Генерируем лицо ребенка** { run: "auto", display-mode: "form" }
#@markdown Доминантные гены:<br>Генотип отца - генотип матери: [0 - 1]
genes_influence = 0.5 #@param {type:"slider", min:0.01, max:0.99, step:0.01}
#@markdown Стилизация фотографии:
style = "По умолчанию" #@param ["По умолчанию", "Фото отца", "Фото матери"]
if style == "Фото отца": 
  lr = ((np.arange(1,model_scale+1)/model_scale)**genes_influence).reshape((model_scale,1))
  rl = 1-lr
  hybrid_face = (lr*first_face) + (rl*second_face)
elif style == "Фото матери": 
  lr = ((np.arange(1,model_scale+1)/model_scale)**(1-genes_influence)).reshape((model_scale,1))
  rl = 1-lr
  hybrid_face = (rl*first_face) + (lr*second_face)
else: hybrid_face = ((1-genes_influence)*first_face)+(genes_influence*second_face)
#@markdown Примерный возраст ребенка:
person_age = 20 #@param {type:"slider", min:10, max:50, step:1}
intensity = -((person_age/5)-6)

#@markdown Скачать итоговое изображение?
download_image = False #@param {type:"boolean"}

if download_image: resolution = "1024"
else: resolution = "256"

size = int(resolution), int(resolution)
face = generate_final_image(hybrid_face, age_direction, intensity)
plot_three_images(face, fs = 15)
if download_image: files.download('face.png')

In [ ]:
#@title # **Изменяем параметры лица ребенка** { run: "auto", display-mode: "form" }
#@markdown (можно поиграться с параметрами если вам не нравится результат)

#@markdown Доминантные гены:<br>Генотип отца - генотип матери: [0 - 1]
genes_influence = 0.5 #@param {type:"slider", min:0.01, max:0.99, step:0.01}
#@markdown Стилизация фотографии:
style = "По умолчанию" #@param ["По умолчанию", "Фото отца", "Фото матери"]
if style == "Фото отца": 
  lr = ((np.arange(1,model_scale+1)/model_scale)**genes_influence).reshape((model_scale,1))
  rl = 1-lr
  hybrid_face = (lr*first_face) + (rl*second_face)
elif style == " Фото матери": 
  lr = ((np.arange(1,model_scale+1)/model_scale)**(1-genes_influence)).reshape((model_scale,1))
  rl = 1-lr
  hybrid_face = (rl*first_face) + (lr*second_face)
else: hybrid_face = ((1-genes_influence)*first_face)+(genes_influence*second_face)

#@markdown Интенсивность опции:
horizontal = 23 #@param {type:"slider", min:-100, max:100, step:1}
horizontal_intensity = horizontal/50
vertical = -80 #@param {type:"slider", min:-100, max:100, step:1}
vertical_intensity = vertical/50
eyes_open = 20 #@param {type:"slider", min:-100, max:100, step:1}
eyes_open_intensity = -(eyes_open/25)
gender = 100 #@param {type:"slider", min:-100, max:100, step:1}
gender_intensity = gender/100
smile = 35 #@param {type:"slider", min:-100, max:100, step:1}
smile_intensity = smile/100
age = 20 #@param {type:"slider", min:-100, max:100, step:1}
age_intensity = -(age/25)

direction_intensity = (horizontal_direction*horizontal_intensity) + (vertical_direction*vertical_intensity) + (eyes_open_direction*eyes_open_intensity) + (gender_direction*gender_intensity) + (smile_direction*smile_intensity) + (age_direction*age_intensity)

#@markdown Скачать итоговое изображение?
download_image = False #@param {type:"boolean"}
#resolution = "1024" #@param [256, 512, 1024]
if download_image: resolution = "1024"
else: resolution = "256"
size = int(resolution), int(resolution)

face = generate_final_image(hybrid_face, direction_intensity, 1)
plot_three_images(face, fs = 15)
if download_image: files.download('face.png')